ANALISIS EDA DETALLADO-2 SPRINT 

DATA FRAME MASTER 

In [12]:
import os

BASE = r"d:\Users\usuario\Desktop\Tablas limpias para EDA\\"
print("📁 Archivos encontrados en la carpeta:\n")
for f in os.listdir(BASE):
    print(f)


📁 Archivos encontrados en la carpeta:

df_master_games.csv
dim_estadio.csv
dim_jugador.csv
dim_team.csv
fact_partidos_2010_2023.csv
play_by_play_limpio.csv


In [13]:
BASE = r"d:\Users\usuario\Desktop\Tablas limpias para EDA\\"

df_partidos = pd.read_csv(BASE + "fact_partidos_2010_2023.csv")
df_teams    = pd.read_csv(BASE + "dim_team.csv")
df_estadio  = pd.read_csv(BASE + "dim_estadio.csv")


In [15]:
# ============================================================
# 2.5 — CÓMO SE CONSTRUYE EL DATAFRAME MAESTRO (fiel a tu plan)
# ============================================================

import numpy as np
import pandas as pd

# ---------- Carga (ya esta cargada) ----------
# BASE = r"d:\Users\usuario\Desktop\Tablas limpias para EDA\\"
# df_partidos = pd.read_csv(BASE + "fact_partidos_2010_2023.csv")
# df_teams    = pd.read_csv(BASE + "dim_team.csv")
# df_estadio  = pd.read_csv(BASE + "dim_estadio.csv")

# ---------- Limpieza y preparación ----------
# (1) Selección de columnas relevantes (como en tu slide)
cols_partidos = [
    'game_id',
    # temporada/fecha: usa 'season' si existe; si no, 'season_id'  y 'game_date_x'
    'season', 'season_id', 'game_date', 'game_date_x',
    'team_id_home', 'team_id_away',
    'pts_home_x', 'pts_away_x', 'wl_home', 'attendance',
    'fgm_home', 'fga_home', 'fta_home', 'oreb_home', 'tov_home',
    'fgm_away', 'fga_away', 'fta_away', 'oreb_away', 'tov_away'
]
cols_partidos = [c for c in cols_partidos if c in df_partidos.columns]
df_part = df_partidos[cols_partidos].copy()

# Normalizo fecha/temporada por si vienen con otros nombres
if 'game_date' not in df_part.columns and 'game_date_x' in df_part.columns:
    df_part['game_date'] = pd.to_datetime(df_part['game_date_x'], errors='coerce')
elif 'game_date' in df_part.columns:
    df_part['game_date'] = pd.to_datetime(df_part['game_date'], errors='coerce')

if 'season' not in df_part.columns:
    if 'season_id' in df_part.columns:
        df_part['season'] = df_part['season_id']
    elif 'game_date' in df_part.columns:
        df_part['season'] = df_part['game_date'].dt.year

# Aseguro tipos para el merge
for c in ['team_id','team_id_home','team_id_away']:
    if c in df_teams.columns:
        df_teams[c] = df_teams[c].astype(str).str.strip()
    if c in df_part.columns:
        df_part[c] = df_part[c].astype(str).str.strip()

# (2) Unificación de nombres de equipos 
equipos = ['Golden State Warriors', 'Oklahoma City Thunder', 'Sacramento Kings']
df_teams3 = df_teams[df_teams['team_name'].isin(equipos)].copy()

# además, filtramos los partidos a juegos donde participe al menos uno de los 3 equipos
ids_target = df_teams3['team_id'].tolist()
df_part = df_part[(df_part['team_id_home'].isin(ids_target)) | (df_part['team_id_away'].isin(ids_target))].copy()

# ---------- Construcción del DataFrame maestro ----------
# (3) Datos del equipo local
df_home = df_part.merge(df_teams3[['team_id','team_name']],
                        left_on='team_id_home', right_on='team_id', how='left')
df_home['is_home']      = 1
df_home['pts_team']     = df_home['pts_home_x']
df_home['pts_opponent'] = df_home['pts_away_x']
df_home['win_flag']     = np.where(df_home['wl_home'] == 'W', 1, 0)

# (4) Datos del equipo visitante
df_away = df_part.merge(df_teams3[['team_id','team_name']],
                        left_on='team_id_away', right_on='team_id', how='left')
df_away['is_home']      = 0
df_away['pts_team']     = df_away['pts_away_x']
df_away['pts_opponent'] = df_away['pts_home_x']
df_away['win_flag']     = np.where(df_away['wl_home'] == 'L', 1, 0)

# (5) Unión de ambos
df_master = pd.concat([df_home, df_away], ignore_index=True)

# ---------- Cálculo de KPIs (como en tu slide, pero correcto para home/away) ----------
# Tu slide calcula posesiones con columnas *_home para todo.
# Para que sea fiel a la idea pero correcto, usamos np.where según is_home:
df_master['possessions'] = np.where(
    df_master['is_home'].eq(1),
    (df_master['fga_home'].fillna(0)
     + 0.44 * df_master['fta_home'].fillna(0)
     - df_master['oreb_home'].fillna(0)
     + df_master['tov_home'].fillna(0)),
    (df_master['fga_away'].fillna(0)
     + 0.44 * df_master['fta_away'].fillna(0)
     - df_master['oreb_away'].fillna(0)
     + df_master['tov_away'].fillna(0))
)

# OffRtg, DefRtg, NetRtg (idéntico a tu slide)
den = df_master['possessions'].replace(0, np.nan)
df_master['off_rtg'] = (df_master['pts_team']     / den) * 100
df_master['def_rtg'] = (df_master['pts_opponent'] / den) * 100
df_master['net_rtg'] = df_master['off_rtg'] - df_master['def_rtg']

# Margen (idéntico a tu slide)
df_master['win_margin'] = df_master['pts_team'] - df_master['pts_opponent']

# ---------- Selección de columnas finales (como sugerencia de tu slide) ----------
cols_finales = [
    'game_id','game_date','season',
    'team_name','is_home',
    'pts_team','pts_opponent','win_flag','win_margin',
    'off_rtg','def_rtg','net_rtg','possessions','attendance'
]
df_master = df_master[[c for c in cols_finales if c in df_master.columns]].copy()

# Guardar
df_master.to_csv(BASE + "df_master_games.csv", index=False)
print(f"✅ df_master creado: {df_master.shape[0]} filas x {df_master.shape[1]} columnas")
df_master.head()


✅ df_master creado: 6586 filas x 14 columnas


,game_id,game_date,season,team_name,is_home,pts_team,pts_opponent,win_flag,win_margin,off_rtg,def_rtg,net_rtg,possessions,attendance
0,20900476,2010-01-01,2009.0,NaN,1,109.0,108.0,1,1.0,113.826232,112.781955,1.044277,95.76,18997.0
1,20900484,2010-01-02,2009.0,NaN,1,103.0,97.0,1,6.0,98.696819,92.947489,5.749329,104.36,15264.0
2,20900488,2010-01-02,2009.0,Sacramento Kings,1,91.0,99.0,0,-8.0,88.590343,96.378505,-7.788162,102.72,14294.0
3,20900487,2010-01-02,2009.0,NaN,1,105.0,89.0,1,16.0,115.435356,97.845207,17.590150,90.96,20507.0
4,20900495,2010-01-04,2009.0,NaN,1,85.0,98.0,0,-13.0,87.592745,100.989283,-13.396538,97.04,18838.0


In [33]:
# Limpieza final 
df_master = df_master.dropna(subset=['team_name']).copy()

print(f"✅ df_master final listo: {df_master.shape[0]} filas x {df_master.shape[1]} columnas")
df_master.head()


✅ df_master final listo: 3456 filas x 34 columnas


,game_id,season,team_id_home,team_id_away,pts_home_x,pts_away_x,wl_home,attendance,fgm_home,fga_home,...,year_founded,is_home,pts_team,pts_opponent,win_flag,possessions,off_rtg,def_rtg,net_rtg,win_margin
10,20900488,"2,009.00",1610612758,1610612742,91.00,99.00,L,"14,294.00",37.00,92.00,...,"1,948.00",1,91.00,99.00,0,102.72,88.59,96.38,-7.79,-8.00
27,20900505,"2,009.00",1610612758,1610612756,109.00,113.00,L,"13,630.00",42.00,94.00,...,"1,948.00",1,109.00,113.00,0,104.88,103.93,107.74,-3.81,-4.00
37,20900512,"2,009.00",1610612760,1610612740,92.00,97.00,L,"17,836.00",35.00,79.00,...,"1,967.00",1,92.00,97.00,0,88.12,104.40,110.08,-5.67,-5.00
54,20900529,"2,009.00",1610612744,1610612758,108.00,101.00,W,"18,327.00",38.00,79.00,...,"1,946.00",1,108.00,101.00,1,101.52,106.38,99.49,6.90,7.00
60,20900537,"2,009.00",1610612758,1610612743,102.00,100.00,W,"14,411.00",38.00,78.00,...,"1,948.00",1,102.00,100.00,1,93.56,109.02,106.88,2.14,2.00


In [34]:
print(f"✅ df_master creado: {df_master.shape[0]} filas x {df_master.shape[1]} columnas")


✅ df_master creado: 3456 filas x 34 columnas


DATAFREAM DF_TEAM_STATS

In [36]:
import numpy as np
import pandas as pd

# 1) Agregado por equipo, temporada y localía + cantidad de partidos
df_team_stats_localia = (
    df_master.groupby(['team_name', 'season', 'is_home'], as_index=False)
    .agg({
        'game_id'   : 'count',   # <- NUEVO: cantidad de juegos
        'win_flag'  : 'mean',
        'off_rtg'   : 'mean',
        'def_rtg'   : 'mean',
        'net_rtg'   : 'mean',
        'win_margin': 'mean',
        'attendance': 'mean'
    })
    .rename(columns={'game_id':'games_n', 'win_flag':'win_rate'})
)

# 2) Consistencia (std del margen)
df_consistency_localia = (
    df_master.groupby(['team_name', 'season', 'is_home'])['win_margin']
    .std()
    .reset_index()
    .rename(columns={'win_margin': 'std_margin'})
)
df_team_stats_localia = df_team_stats_localia.merge(
    df_consistency_localia,
    on=['team_name', 'season', 'is_home'],
    how='left'
)

# 3) Blindaje: si std_margin es 0/NaN, usa mediana por temporada (evita inf/NaN en IPS)
median_std_by_season = df_team_stats_localia.groupby('season')['std_margin'].transform('median')

df_team_stats_localia['std_margin'] = np.where(
    df_team_stats_localia['std_margin'].isna() | (df_team_stats_localia['std_margin'] == 0),
    median_std_by_season,
    df_team_stats_localia['std_margin']
)


# 4) IPS (fórmula original)
std_safe = df_team_stats_localia['std_margin'].replace(0, np.nan)
df_team_stats_localia['IPS'] = (
    0.4 * df_team_stats_localia['win_rate'] +
    0.3 * df_team_stats_localia['net_rtg'] +
    0.3 * (1 / std_safe)
)

# 5) Limpieza de infinitos y etiqueta de localía (para BI)
df_team_stats_localia.replace([np.inf, -np.inf], np.nan, inplace=True)
df_team_stats_localia['venue'] = np.where(df_team_stats_localia['is_home'].eq(1), 'Local', 'Visitante')

# 6) Columnas ordenadas + export
df_team_stats_localia = df_team_stats_localia[
    ['team_name','season','is_home','venue','games_n',
     'win_rate','off_rtg','def_rtg','net_rtg','win_margin','attendance',
     'std_margin','IPS']
].copy()

# (Opcional) filtrar muestras muy chicas
# df_team_stats_localia = df_team_stats_localia[df_team_stats_localia['games_n'] >= 10]

# Guardar
try:
    BASE
except NameError:
    BASE = "./"
out_path = BASE + "df_team_stats_localia.csv"
df_team_stats_localia.to_csv(out_path, index=False)
print(f"✅ Exportado: {out_path} ({df_team_stats_localia.shape[0]} filas, {df_team_stats_localia.shape[1]} columnas)")
df_team_stats_localia.head()


✅ Exportado: d:\Users\usuario\Desktop\Tablas limpias para EDA\\df_team_stats_localia.csv (84 filas, 13 columnas)


,team_name,season,is_home,venue,games_n,win_rate,off_rtg,def_rtg,net_rtg,win_margin,attendance,std_margin,IPS
0,Golden State Warriors,"2,009.00",0,Visitante,21,0.19,105.64,114.34,-8.70,-8.76,"16,719.67",12.10,-2.51
1,Golden State Warriors,"2,009.00",1,Local,26,0.38,107.57,107.78,-0.21,-0.19,"17,945.04",13.79,0.11
2,Golden State Warriors,"2,010.00",0,Visitante,43,0.21,101.20,110.07,-8.87,-8.74,"16,563.60",11.02,-2.55
3,Golden State Warriors,"2,010.00",1,Local,42,0.67,108.80,104.32,4.48,4.48,"18,117.95",14.38,1.63
4,Golden State Warriors,"2,011.00",0,Visitante,32,0.34,101.34,105.46,-4.12,-4.00,"16,172.34",12.84,-1.08
